# Agent
- 作者： [chaofa用代码打点酱油](https://bruceyuan.com/)，全平台同名
    - [公众号](https://bruceyuan.com/llms-zero-to-hero/chaofa-wechat-official-account.png)/视频号/[YouTube](https://www.youtube.com/@bbruceyuan)/小红书/Twitter

## 1. Function Call 的基础知识

In [1]:
from openai import OpenAI
from dotenv import dotenv_values
import json

key_value_env = dotenv_values(".env")
# print(key_value_env)

In [2]:


client = OpenAI(
    base_url="https://api.deepseek.com/v1",
    api_key=key_value_env["API_KEY"]
)

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "获取当前城市的天气信息",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "城市名字 e.g. 北京"
                },
                "country": {
                    "type": "string",
                    "description": "国家名字 e.g. 中国"
                }
            },
            "required": [
                "location", "coutry"
            ],
            "additionalProperties": False
        }   
    }
},
]


messages =[{"role": "user", "content": "深圳的天气怎么样?"}]

# step 1: 获取 function call 的结果
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
    tools=tools,
    tool_choice="auto", # auto, required, none
)

print(response)

ChatCompletion(id='0de36954-c060-40cc-ba53-66cf1b0785c7', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0_e7d57cac-d0e1-4069-9ab9-82d12a7b372c', function=Function(arguments='{"location":"深圳","country":"中国"}', name='get_weather'), type='function', index=0)]))], created=1744549828, model='deepseek-chat', object='chat.completion', service_tier=None, system_fingerprint='fp_3d5141a69a_prod0225', usage=CompletionUsage(completion_tokens=23, prompt_tokens=158, total_tokens=181, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0), prompt_cache_hit_tokens=0, prompt_cache_miss_tokens=158))


In [3]:
print(response.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_0_e7d57cac-d0e1-4069-9ab9-82d12a7b372c', function=Function(arguments='{"location":"深圳","country":"中国"}', name='get_weather'), type='function', index=0)]


In [4]:
## step3: 执行 function call

tool_call = response.choices[0].message.tool_calls[0]
tool_name = tool_call.function.name
tool_args = json.loads(tool_call.function.arguments)

print(tool_name, )
print(tool_args)

get_weather
{'location': '深圳', 'country': '中国'}


In [5]:
print(tool_call.id)

call_0_e7d57cac-d0e1-4069-9ab9-82d12a7b372c


In [6]:


def get_weather(location, country):
    return (f"{location} 的天气是：晴天")

function_call_result=get_weather(tool_args["location"], tool_args["country"])
print(function_call_result)


深圳 的天气是：晴天


In [7]:
# step 4: 将 function call 的结果返回给 LLM
mesages = messages.append(response.choices[0].message)

messages.append({
    "role": "tool",
    "content": function_call_result,
    "tool_call_id": tool_call.id
})


In [8]:
## step5: 再次调用 LLM


res = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
)

print(res)
print(res.choices[0].message.content)


ChatCompletion(id='7fadab18-4364-4db8-9a55-44ea66f43c96', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='深圳目前是晴天，适合外出活动。如果需要更详细的天气信息，比如温度或湿度，可以告诉我哦！', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1744549833, model='deepseek-chat', object='chat.completion', service_tier=None, system_fingerprint='fp_3d5141a69a_prod0225', usage=CompletionUsage(completion_tokens=24, prompt_tokens=43, total_tokens=67, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0), prompt_cache_hit_tokens=0, prompt_cache_miss_tokens=43))
深圳目前是晴天，适合外出活动。如果需要更详细的天气信息，比如温度或湿度，可以告诉我哦！


## 2. 训练一个自己的 function Call 模型

In [9]:
from enum import Enum
from functools import partial
import pandas as pd
import torch
import json

from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, TaskType

seed = 42
set_seed(seed)

import os

In [10]:
"""
这里主要讲解如何处理数据，
SFT 训练由于前期的视频讲过，因此不过多赘述，可以参考
视频：https://www.bilibili.com/video/BV1NM1tY3Eu5/
代码： https://github.com/bbruceyuan/Hands-On-Large-Language-Models-CN/tree/master/chapter12
"""

'\n这里主要讲解如何处理数据，\nSFT 训练由于前期的视频讲过，因此不过多赘述，可以参考\n视频：https://www.bilibili.com/video/BV1NM1tY3Eu5/\n代码： https://github.com/bbruceyuan/Hands-On-Large-Language-Models-CN/tree/master/chapter12\n'

In [28]:
import subprocess
import os

# 这是 aistackdc 用于 Github/huggingface 下载加速的方式
result = subprocess.run('bash -c "source /etc/network/turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value


model_name = "Qwen/Qwen2.5-0.5B-Instruct"
dataset_name = "Jofthomas/hermes-function-calling-thinking-V1"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 备注：这里的 model 
tokenizer.chat_template = "{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{{ '<|im_start|>' + message['role'] + '\n' + message['content'] | trim + '<|im_end|>\n' }}{% endfor %}{% if add_generation_prompt %}{{'<|im_start|>assistant\n'}}{% endif %}"


def preprocess(sample):
    messages = sample["messages"]
    first_message = messages[0]

    # Instead of adding a system message, we merge the content into the first user message
    if first_message["role"] == "system":
        system_message_content = first_message["content"]
        # Merge system content with the first user message
        messages[1]["content"] = system_message_content + "Also, before making a call to a function take the time to plan the function to take. Make that thinking process between <think>{your thoughts}</think>\n\n" + messages[1]["content"]
        # Remove the system message from the conversation
        messages.pop(0)

    return {"text": tokenizer.apply_chat_template(messages, tokenize=False)}


dataset = load_dataset(dataset_name)
dataset = dataset.rename_column("conversations", "messages")

In [29]:
dataset["train"][0]

{'messages': [{'content': "You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'get_stock_price', 'description': 'Get the current stock price of a company', 'parameters': {'type': 'object', 'properties': {'company': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company']}}}, {'type': 'function', 'function': {'name': 'get_movie_details', 'description': 'Get details about a movie', 'parameters': {'type': 'object', 'properties': {'title': {'type': 'string', 'description': 'The title of the movie'}}, 'required': ['title']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Ar

In [37]:
# 改成和 qwen 对应 assistant / user 的格式
def convert_model_to_assistant(sample):
    messages = sample["messages"]
    for message in messages:
        if message["role"] == "model":
            message["role"] = "assistant"
        if message["role"] == "human":
            message["role"] = "user"
    return sample

dataset = dataset.map(convert_model_to_assistant)


Map:   0%|          | 0/3570 [00:00<?, ? examples/s]

In [36]:
dataset['train'][0]

{'messages': [{'content': "You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'get_stock_price', 'description': 'Get the current stock price of a company', 'parameters': {'type': 'object', 'properties': {'company': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company']}}}, {'type': 'function', 'function': {'name': 'get_movie_details', 'description': 'Get details about a movie', 'parameters': {'type': 'object', 'properties': {'title': {'type': 'string', 'description': 'The title of the movie'}}, 'required': ['title']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Ar

In [13]:
dataset = dataset.map(preprocess, remove_columns="messages")
dataset = dataset["train"].train_test_split(0.1)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 3213
    })
    test: Dataset({
        features: ['text'],
        num_rows: 357
    })
})


In [14]:
print(dataset["train"][8]["text"])

<|im_start|>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'get_news_headlines', 'description': 'Get the latest news headlines', 'parameters': {'type': 'object', 'properties': {'country': {'type': 'string', 'description': 'The country for which headlines are needed'}}, 'required': ['country']}}}, {'type': 'function', 'function': {'name': 'search_recipes', 'description': 'Search for recipes based on ingredients', 'parameters': {'type': 'object', 'properties': {'ingredients': {'type': 'array', 'items': {'type': 'string'}, 'description': 'The list of ingredients'}}, 'required': ['ingredients']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type

In [15]:
# Sanity check
print(tokenizer.pad_token)
print(tokenizer.eos_token)

<|endoftext|>
<|im_end|>


In [16]:
class ChatmlSpecialTokens(str, Enum):
    tools = "<tools>"
    eotools = "</tools>"
    think = "<think>"
    eothink = "</think>"
    tool_call="<tool_call>"
    eotool_call="</tool_call>"
    tool_response="<tool_reponse>"
    eotool_response="</tool_reponse>"
    pad_token = "<|endoftext|>"
    eos_token = "<|im_end|>"
    @classmethod
    def list(cls):
        return [c.value for c in cls]

tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        pad_token=ChatmlSpecialTokens.pad_token.value,
        additional_special_tokens=ChatmlSpecialTokens.list()
    )
tokenizer.chat_template = "{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{{ '<|im_start|>' + message['role'] + '\n' + message['content'] | trim + '<|im_end|>\n' }}{% endfor %}{% if add_generation_prompt %}{{'<|im_start|>assistant\n'}}{% endif %}"



现在我们得到了可以使用的 训练数据，接下来就按照：https://github.com/bbruceyuan/Hands-On-Large-Language-Models-CN/tree/master/chapter12 中的代码，就可以做 SFT 训练，然后模型就可以调用工具。

## ReAct 的实现

In [17]:
# user: input
# model: 一段话 + tool_call
# role: tool + tool_result
# model: 结果

In [18]:

# Prepare system message with tool information
system_message = f"""Answer the following questions as best you can. You have access to the following tools:

{json.dumps(tools)}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be function name
Action Input: the input to the action. eg. {{"param1": "value1"}}
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!"""

react_messages = [{"role": "system", "content": system_message}]
react_messages.append({"role": "user", "content": "帮我查看一下深圳的天气怎么样？"})
        
# Call LLM API
completion = client.chat.completions.create(
        model="deepseek-chat",
        messages=react_messages,
        temperature=0.3,
        stop="Observation:"
)
print(completion.choices[0].message.content)

Thought: 用户想要了解深圳的天气情况，我需要调用天气查询功能来获取深圳的天气信息。

Action: get_weather
Action Input: {"location": "深圳", "country": "中国"}
